<h1 align=center><font size = 5>Clustering Neighborhoods in Toronto</font></h1>

Firs we do all required imports and we install beautifulsoup4

In [1]:
import requests
import pandas as pd
!pip install beautifulsoup4
from bs4 import BeautifulSoup

<br>
<br>
Now we perform the request to the Wikipedia website to get all html content

In [2]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641"
page = requests.get(url)
page

<Response [200]>

<br>
<br>
We have to find the content of the table we want to clean

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find_all('table')[0]
data = []

for child in soup.find_all('table')[0].children:
    for td in child:
        if isinstance(td, str) == False:
            data.append(td.text.split('\n'))        
for i in range(len(data)):
    while("" in data[i]) :
        data[i].remove("")

<br>
<br>
We now build our DataFrame with the list "data" that contains all the required information 

In [4]:
df = pd.DataFrame(data[1::]) #skipping first row to use as columns names
df.columns = data[0]
df.shape

(288, 3)

<br>
<br>
-Ignoring cells with a borough that is Not assigned.<br>
-If cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.<br>
-If more than one neighborhood exist in one postal code area. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [5]:
df = df.loc[df['Borough']!='Not assigned']
df['Neighbourhood'].replace('Not assigned',df[df['Neighbourhood']=='Not assigned']['Borough'].values[0],inplace=True)
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
print(df.to_string())

    Postcode           Borough                                                                                                                    Neighbourhood
0        M1B       Scarborough                                                                                                                    Rouge,Malvern
1        M1C       Scarborough                                                                                             Highland Creek,Rouge Hill,Port Union
2        M1E       Scarborough                                                                                                  Guildwood,Morningside,West Hill
3        M1G       Scarborough                                                                                                                           Woburn
4        M1H       Scarborough                                                                                                                        Cedarbrae
5        M1J       Scarborough          

In [6]:
df.shape

(103, 3)